In [1]:
# Import packages

import numpy as np
import matplotlib.pyplot as plt
from netCDF4 import Dataset
from matplotlib import colors
import cartopy.crs as ccrs
import datetime as dt
import glob, os, sys
from matplotlib import animation
import matplotlib.path as mpath
import cartopy.feature as cfeature
import re
import os
import pandas as pd
import seaborn as sns
import math
import netCDF4 as nc

In [2]:
base_path = '/gws/nopw/j04/asci/yixin/ftp.archive.arm.gov/dingy1/'# the path contains all instrument measurement 

# set empty list and dictionary
filelist = []
filedic = {}
numdic={}

#define time period
begin = 20161020
end = 20170224

#Reads the list of files and folders in the specified path
second_path = os.listdir(base_path)
for second in second_path:
    if '.' not in second:
        filedir = os.path.join(base_path,second)
        files = os.listdir(filedir)
        
        # use regular expressions to find number string indicate the measurment date
        for file in files:
            find_number = re.compile("\d{8}") 
            number = re.findall(find_number,file)
            
            # compare number string with target time period 
            if number:
                number = number[0]                
                if begin<=int(number)<=end and not filedic.get(file, False) and not numdic.get(number, False) :
                    filelist.append(os.path.join(filedir,file))
                    

In [3]:
# set empty list
wacr_list = []
# use loop to append the file of target VAP
for i in filelist:
    if 'asiarsclwacr1kolliasM1'in i:
        wacr_list.append(i)
        
wacr_list.sort() # sort in date order
print('wacr size is',len(wacr_list))
print(wacr_list[0],wacr_list[-1]) #print first and last datafile to check the measurment time

wacr size is 145
/gws/nopw/j04/asci/yixin/ftp.archive.arm.gov/dingy1/229920/asiarsclwacr1kolliasM1.c1.20161020.190000.custom.nc /gws/nopw/j04/asci/yixin/ftp.archive.arm.gov/dingy1/229920/asiarsclwacr1kolliasM1.c1.20170224.000000.custom.nc


In [37]:
# make empty arrays
wacr_ref,wacr_time= np.array([]),np.array([])

for f in wacr_list:
    
    nc_wacr = Dataset(f,'r')

    ref = nc_wacr.variables['reflectivity_best_estimate'][:]
    height = nc_wacr.variables['height'][:] # indicate the each layer height of reflectivity measurement
    cloud = nc_wacr.variables['cloud_base_best_estimate'][:]
    cloud_qc = nc_wacr.variables['qc_cloud_base_best_estimate'][:]
    wacr_data_time,time_unit=nc_wacr.variables['time'][:],nc_wacr.variables['time'].units
    data_time=nc.num2date(wacr_data_time,units=time_unit) # Convert the time format

    # extract data with good quality 
    cloud_base = cloud[(cloud_qc==0) & (cloud<=10000)]
    data_time = data_time[(cloud_qc==0) & (cloud<=10000)]
    ref = ref[(cloud_qc==0) & (cloud<=10000)]

    # make empty list to store the reflectivity measurement height corresponding to the cloud base height
    index=list()
    # Look for cloud base heights to fall within which range of reflectivity measurement heights, and append this measurement heights index
    for i in np.arange(cloud_base.shape[0]):
        for j in np.arange(len(height)):
            if cloud_base[i]>=height[j] and cloud_base[i]<height[j+1]:
                index.append(j)
                
    # according to the reflectivity measurement height index, take slice of reflectivity
    for i in np.arange(len(index)):
        ref_slice=ref[:,index[i]]
        
    #append time and slice of reflectivity array
    wacr_ref = np.append(wacr_ref,ref_slice)
    wacr_time= np.append(wacr_time,data_time)
    


In [49]:
print(wacr_ref.shape)
print(wacr_time.shape)

(2299778,)
(2299778,)


In [50]:
# Combine time and refliectivety data 
ref_array=np.stack((wacr_time, wacr_ref), axis=1) 
# Convert NumPy Array to Pandas DataFrame
ref_df = pd.DataFrame(ref_array)
# define the colums name
ref_df.columns = ['time','ref']
# delete bad data
ref_df = ref_df.drop(ref_df[(ref_df['ref']== -9999.0)].index,axis = 0)
# Store the data in a new CSV file
ref_df.to_csv("time_ref_full.csv",index=False,sep=',')
